<a href="https://colab.research.google.com/github/sirpps/AICoverGen/blob/main/AiCoverGenCLI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Clone and install that shi
import codecs
import os
from IPython.display import clear_output as cls
#==========Cloning rep==================
!rm -rf sample_data
if os.path.exists("/content/eai") and os.path.isdir("/content/eai"):
    print("Already cloning, skipping")
else:
    clrep=codecs.decode('uggcf://tvguho.pbz/fveccf/NVPbireTra', 'rot_13')
    !git clone $clrep eai
    %cd eai/eserce
    cls()
    print("Finsihed Cloning, starting install requirements")
#==========Install y====================
!pip install pip==24.0
!pip install --no-cache-dir fairseq==0.12.2 faiss-cpu==1.7.3 omegaconf ffmpeg-python>=0.2.0 gradio==3.39.0 numpy==1.26.4 onnxruntime_gpu==1.20.0 praat-parselmouth>=0.4.2 pedalboard==0.7.7 pyworld==0.3.4 torchcrepe==0.0.20 yt_dlp==2024.11.04 sox==1.4.1
# install cuda fix
!python -m pip install ort-nightly-gpu --index-url=https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/ort-cuda-12-nightly/pypi/simple/
!sudo apt install sox -y -qq
cls()
# Download models
models=codecs.decode('fep/qbjaybnq_zbqryf.cl', 'rot_13')
!python $models
cls()
print("Finished Downloading")
#============Debug======================
#!python -c "import os; vpip=int(os.system('pip --version')); print(vpip)"
#!python -c "import numpy; print('numpy version: '+numpy.__version__)"
#!python -c "import os; vsox=int(os.system('sox --version')); print(vsox)"

In [ ]:
#@title Model Download Function

import os
import zipfile
import shutil
import urllib.request

BASE_DIR = os.getcwd()
rvc_models_dir = os.path.join(BASE_DIR, 'rvc_models')

def extract_zip(extraction_folder, zip_name):
    os.makedirs(extraction_folder)
    with zipfile.ZipFile(zip_name, 'r') as zip_ref:
        zip_ref.extractall(extraction_folder)
    os.remove(zip_name)

    index_filepath, model_filepath = None, None
    for root, dirs, files in os.walk(extraction_folder):
        for name in files:
            if name.endswith('.index') and os.stat(os.path.join(root, name)).st_size > 1024 * 100:
                index_filepath = os.path.join(root, name)

            if name.endswith('.pth') and os.stat(os.path.join(root, name)).st_size > 1024 * 1024 * 40:
                model_filepath = os.path.join(root, name)

    if not model_filepath:
        raise Exception(f'No .pth model file was found in the extracted zip. Please check {extraction_folder}.')

    # move model and index file to extraction folder
    os.rename(model_filepath, os.path.join(extraction_folder, os.path.basename(model_filepath)))
    if index_filepath:
        os.rename(index_filepath, os.path.join(extraction_folder, os.path.basename(index_filepath)))

    # remove any unnecessary nested folders
    for filepath in os.listdir(extraction_folder):
        if os.path.isdir(os.path.join(extraction_folder, filepath)):
            shutil.rmtree(os.path.join(extraction_folder, filepath))

def download_online_model(url, dir_name):
    try:
        print(f'[~] Downloading voice model with name {dir_name}...')
        zip_name = url.split('/')[-1]
        extraction_folder = os.path.join(rvc_models_dir, dir_name)
        if os.path.exists(extraction_folder):
            raise Exception(f'Voice model directory {dir_name} already exists! Choose a different name for your voice model.')

        if 'pixeldrain.com' in url:
            url = f'https://pixeldrain.com/api/file/{zip_name}'

        urllib.request.urlretrieve(url, zip_name)

        print('[~] Extracting zip...')
        extract_zip(extraction_folder, zip_name)
        print(f'[+] {dir_name} Model successfully downloaded!')

    except Exception as e:
        raise Exception(str(e))

# @markdown Enter models direct link, pixeldrain supproted
url = "" # @param {type:"string"}
# @markdown Anything
dir_name = "" # @param {type:"string"}

download_online_model(url, dir_name)

In [ ]:
# @title Generate
 # old@title Generate | Output generated inside "AICoverGen\song_output\random_number"
# old@markdown Main Option | You also can input audio path inside "SONG_INPUT"
# @markdown Enter youtube link only
SONG_INPUT = "" # @param {type:"string"}
RVC_DIRNAME = dir_name # [@param] {type:"string"}
# @markdown Use 1 to 12 for female sound, -1 to -12 for male sound<br>(min: -12.0, max: 12.0)<br>
# Read by yours, don't lazy. Eg, your model is girl and song input is man voice, so you add positive int<br>or your model is man and song input girl voice, so you add negative int
PITCH_CHANGE = 0 # @param {type:"integer"}
# @markdown Change pitch of vocals and instrumental<br>(min: -12.0, max: 12.0)
PITCH_CHANGE_ALL = 0 # @param {type:"integer"}
# @markdown ---
# @markdown ### Voice Conversion Options
# @markdown Controls how many of Ai voice's accent to keep in the vocal<br>(min: 0.x, max: 1.0)
INDEX_RATE = 0.85 # @param {type:"number"}
# @markdown If >=3, Reduce breathiness<br>(min: 0.x, max: 7.0)
FILTER_RADIUS = 7 # @param {type:"integer"}
PITCH_DETECTION_ALGO = "rmvpe" # [@param] ["rmvpe", "mangio-crepe"]
CREPE_HOP_LENGTH = 128 # [@param] {type:"integer"}
# @markdown Protect voiceless and breath sounds, 0.5 to disable<br>(min: 0.x, max: 0.5)
PROTECT = 0.27 # @param {type:"number"}
# @markdown Control how much mimic the original vocals<br>(min: 0.x, max: 1.0)
REMIX_MIX_RATE = 0.25  # @param {type:"number"}
# @markdown ---
# @markdown ### Audio Mixing Options
MAIN_VOL = 0 # @param {type:"integer"}
BACKUP_VOL = 0 # @param {type:"integer"}
INST_VOL = 0 # @param {type:"integer"}
# @markdown ---
# @markdown ### Reverb Control
# @markdown Overall just change the reverb_size<br>(min: 0.x, max: 1.0)
REVERB_SIZE = 0.18 # @param {type:"number"}
REVERB_WETNESS = 0.2 # @param {type:"number"}
REVERB_DRYNESS = 0.8 # @param {type:"number"}
REVERB_DAMPING = 0.7 # @param {type:"number"}
# @markdown ---
# @markdown ### Output
OUTPUT_FORMAT = "mp3" # @param ["mp3", "wav"]
# @markdown wav higher quality😁, bigger file size🗿<br>mp3 lower quality🗿, lower file size😁
from IPython.display import Audio, display
from urllib.parse import urlparse
import os, re, yt_dlp, subprocess, json, librosa, time
import numpy as np
command = [
    "python",
    "/content/eai/eserce/src/main.py",
    "-i", SONG_INPUT,
    "-dir", RVC_DIRNAME,
    "-p", str(PITCH_CHANGE),
    "-k",
    "-ir", str(INDEX_RATE),
    "-fr", str(FILTER_RADIUS),
    "-rms", str(REMIX_MIX_RATE),
    "-palgo", PITCH_DETECTION_ALGO,
    "-hop", str(CREPE_HOP_LENGTH),
    "-pro", str(PROTECT),
    "-mv", str(MAIN_VOL),
    "-bv", str(BACKUP_VOL),
    "-iv", str(INST_VOL),
    "-pall", str(PITCH_CHANGE_ALL),
    "-rsize", str(REVERB_SIZE),
    "-rwet", str(REVERB_WETNESS),
    "-rdry", str(REVERB_DRYNESS),
    "-rdamp", str(REVERB_DAMPING),
    "-oformat", OUTPUT_FORMAT
]

# Open a subprocess and capture its output
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)

# Print the output in real-time
for line in process.stdout:
    print(line, end='')

# Wait for the process to finish
process.wait()
time.sleep(3)
cls()
#===========biar gampang wak======
url = SONG_INPUT
penyanyi = RVC_DIRNAME
format = OUTPUT_FORMAT
rate = 44100
#===========def===================
def get_youtube_video_id(url, ignore_playlist=True):
    query = urlparse(url)
    if query.hostname == 'youtu.be':
        if query.path[1:] == 'watch':
            return query.query[2:]
        return query.path[1:]

    if query.hostname in {'www.youtube.com', 'youtube.com', 'music.youtube.com'}:
        if not ignore_playlist:
            # use case: get playlist id not current video in playlist
            with suppress(KeyError):
                return parse_qs(query.query)['list'][0]
        if query.path == '/watch':
            return parse_qs(query.query)['v'][0]
        if query.path[:7] == '/watch/':
            return query.path.split('/')[1]
        if query.path[:7] == '/embed/':
            return query.path.split('/')[2]
        if query.path[:3] == '/v/':
            return query.path.split('/')[2]

    # returns None for invalid YouTube url
    return None
vidtitel = subprocess.run(
    ['yt-dlp', '--dump-json', url],
    capture_output=True,
    text=True
)
#===========main code============
video_id = get_youtube_video_id(url)
video_title = json.loads(vidtitel.stdout)['title']
output_folder = os.path.join("/content/eai/eserce/song_output/", video_id)
file_name = f"{video_title} ({penyanyi} Ver).{format}"
path_file = os.path.join(output_folder, file_name)
os.chdir(output_folder)
#===========display to cell=======
try:
#unused but sometimes important   display(Audio(path_file, rate=rate, autoplay=True))
#unused but sometimes important   final, rate = librosa.load(file_name, sr=None)
    print("Your audio name: "+file_name)
    print("Enjoy")
    display(Audio(file_name))
except FileNotFoundError:
    print("An error occured")
#===========debug=================
#print("file name: "+file_name)
#print("file path: "+path_file)
#!ls